In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import plotly
plotly.tools.set_credentials_file(username='quoniammm', api_key='IF7kV6idFRdoo7LdgGRp')
import plotly.plotly as py
import plotly.graph_objs as go
from wordcloud import WordCloud, STOPWORDS
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split

import nltk

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable

%matplotlib inline

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample = pd.read_csv('sample_submission.csv')

In [2]:
# bag_words
all_words = set(train['text'].str.split(expand=True).unstack())

def wordandindex(vocab):
    return {word: i + 3 for i, word in enumerate(vocab)}, {i + 3: word for i, word in enumerate(vocab)}

word2index, index2word = wordandindex(all_words)

label_encoder = LabelEncoder()
# 数据集准备
X = np.array(train.text.apply(lambda sen: [word2index[word] for word in sen.split(' ')]))
y = np.array(label_encoder.fit_transform(train.author))
assert len(X) == len(y)
print(len(all_words))
#print(len(train))
print("test length: {}".format(len(test)))
# 句子填充
X_pad = np.zeros((19579, 861))

for i in range(X_pad.shape[0]):
    for j in range(len(X[i])):
        X_pad[i, j] = X[i][j]

xtrain, xvalid, ytrain, yvalid = train_test_split(
    X_pad, y, 
    stratify=y, 
    random_state=42, 
    test_size=0.1, 
    shuffle=True
)
print("train length: {}".format(len(xtrain)))
#print(xtrain.type)
print("valid length: {}".format(len(xvalid)))
#print(xvalid.type)

# 最长句子长度设置为 input_size
max = 0
for i, x in enumerate(X):
    # print(len(x))
    if len(x) > max:
        max = len(x)
        
print(max)
# train.iloc[9215].values

# 参数
input_size = 861
num_classes = 3
epochs = 30
lr = 1e-

47557
test length: 8392
train length: 17621
valid length: 1958
861


In [3]:
# 逻辑回归
class LR(nn.Module):
    def __init__(self, input_size, num_classes):
        super(LR, self).__init__()
        self.linear_0 = nn.Linear(input_size, 8192)
        self.linear_1 = nn.Linear(8192, 4096)
        self.linear_2 = nn.Linear(4096, 2048)
        self.linear_3 = nn.Linear(2048, 512)
        self.linear_4 = nn.Linear(512, 128)
        self.linear_5 = nn.Linear(128, 64)
        self.linear_6 = nn.Linear(64, 32)
        self.linear_7 = nn.Linear(32, num_classes)
        
    
    def forward(self, x):
        L1_out = F.elu(self.linear_0(x))
        L2_out = F.elu(self.linear_1(L1_out))
        L3_out = F.elu(self.linear_2(L2_out))
        L4_out = F.elu(self.linear_3(L3_out))
        L5_out = F.elu(self.linear_4(L4_out))
        L6_out = F.elu(self.linear_5(L5_out))
        L7_out = F.elu(self.linear_6(L6_out))
        
        
        final_out = F.log_softmax(self.linear_7(L7_out), dim=1)
        logits = F.softmax(self.linear_7(L7_out), dim=1)
        
        return final_out, logits
    
model = LR(input_size, num_classes)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.NLLLoss()

In [4]:
model.cuda()

LR(
  (linear_0): Linear(in_features=861, out_features=8192)
  (linear_1): Linear(in_features=8192, out_features=4096)
  (linear_2): Linear(in_features=4096, out_features=2048)
  (linear_3): Linear(in_features=2048, out_features=512)
  (linear_4): Linear(in_features=512, out_features=128)
  (linear_5): Linear(in_features=128, out_features=64)
  (linear_6): Linear(in_features=64, out_features=32)
  (linear_7): Linear(in_features=32, out_features=3)
)

In [5]:
# 训练模型
train_acc = []
valid_acc = []
for epoch in range(epochs):
    vx = Variable(torch.FloatTensor(xtrain)).cuda()
    vy = Variable(torch.LongTensor(ytrain)).cuda()
    vx_valid = Variable(torch.FloatTensor(xvalid)).cuda()
    vy_valid = Variable(torch.LongTensor(yvalid)).cuda()
    losss = []
    optimizer.zero_grad()
    for i in range(0, len(xtrain), 256):
        if i + 256 > len(xtrain):
            vx_batch = vx[i:len(xtrain)]
            vy_batch = vy[i:len(xtrain)]
        else:
            vx_batch = vx[i:i+256]
            vy_batch = vy[i:i+256]
            
        
        outputs, results = model(vx_batch)
        
        if not i % 25600:
            _, ressults_valid = model(vx_valid)
            _, res = results.data.max(1)
            _, res_valid = ressults_valid.data.max(1)
            ta = (torch.sum(res == vy_batch.data) + 0.0) / 256.0
            va = (torch.sum(res_valid == vy_valid.data) + 0.0) / len(yvalid)
            train_acc.append(ta)
            valid_acc.append(va)
        
        loss = criterion(outputs, vy_batch)
        loss.backward()
        losss.append(loss)
        optimizer.step()
        
    print(loss.data[0])

6.354103088378906
2.432854652404785
1.5135761499404907
1.1455076932907104
1.113584041595459
1.1097564697265625
1.095628023147583
1.1014715433120728
1.0897088050842285
1.1368136405944824
1.1015968322753906
1.1070098876953125
1.1029592752456665
1.1216243505477905
1.0948398113250732
1.165429711341858
1.1039444208145142
1.1403807401657104
1.178102970123291
1.1580079793930054
1.099636435508728
1.257710337638855
1.2039588689804077
1.1918848752975464
1.138580083847046
1.1326384544372559
1.1586968898773193
1.1722626686096191
1.23837411403656
1.184097409248352


In [6]:
N = len(train_acc)
print(N)
random_x = np.linspace(0, 1, N)
random_y0 = np.array(train_acc)
random_y1 = np.array(valid_acc)

# Create traces
trace0 = go.Scatter(
    x = random_x,
    y = random_y0,
    mode = 'lines',
    name = 'train_acc'
)
trace1 = go.Scatter(
    x = random_x,
    y = random_y1,
    mode = 'lines',
    name = 'valid_acc'
)

data = [trace0, trace1]

py.iplot(data, filename='line-mode')

30


In [13]:
def test_deal(x):
    sen_vec = []
    for word in x.split(' '):
        if word not in all_words:
            sen_vec.append(2)
        else:
            sen_vec.append(word2index[word])
            
    return sen_vec
            
X_t = np.array(test.text.apply(test_deal))

X_t_pad = np.zeros((8392, 861))

for i in range(X_t_pad.shape[0]):
    for j in range(len(X_t[i])):
        X_t_pad[i, j] = X_t[i][j]
        
tt = Variable(torch.FloatTensor(X_t_pad))
model.cpu()
_, res = model(tt)
res

Variable containing:
 0.6519  0.2730  0.0751
 0.4462  0.2754  0.2785
 0.2418  0.4984  0.2598
           ⋮            
 0.2638  0.4933  0.2429
 0.5463  0.1870  0.2667
 0.4335  0.1428  0.4237
[torch.FloatTensor of size 8392x3]

In [14]:
rr = pd.DataFrame(np.array(res.data), columns=['EAP', 'HPL', 'MWS'], index=test.id)

In [15]:
rr.head()

,EAP,HPL,MWS
id,,,
id02310,0.651887,0.273037,0.075077
id24541,0.446181,0.275357,0.278462
id00134,0.241819,0.498354,0.259826
id27757,0.284279,0.444009,0.271712
id04081,0.665469,0.213206,0.121325


In [16]:
sample.head()

,id,EAP,HPL,MWS
0,id02310,0.403494,0.287808,0.308698
1,id24541,0.403494,0.287808,0.308698
2,id00134,0.403494,0.287808,0.308698
3,id27757,0.403494,0.287808,0.308698
4,id04081,0.403494,0.287808,0.308698


In [17]:
rr.to_csv('222.csv')